In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from skimage.segmentation import morphological_chan_vese
from skimage.filters import gaussian
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

IMAGES_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Images"
MASK_FOLDER = "/kaggle/input/glaucoma-datasets/G1020/Masks"
df = pd.read_csv("/kaggle/input/glaucoma-datasets/G1020/G1020.csv")

def load_data(df, image_size=(512, 512)):
    images, labels = [], []
    
    for _, row in df.iterrows():
        img_name, label = row['imageID'], row['binaryLabels']
        img_path = os.path.join(IMAGES_FOLDER, img_name)
        
        if not os.path.exists(img_path):
            continue

        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, image_size)
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        edges = cv2.Laplacian(gray, cv2.CV_64F)
        edges = cv2.convertScaleAbs(edges) 
        sharpened = cv2.addWeighted(gray, 1.5, edges, -0.5, 0)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(sharpened)
        normalized = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)
        
        images.append(normalized)
        labels.append(label)
    
    return np.array(images), np.array(labels)


def segment_optic_disc(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    smoothed = gaussian(gray, sigma=1)
    segmented_mask = morphological_chan_vese(smoothed, iterations=100, init_level_set="checkerboard")
    return segmented_mask

def glcm_features(image):
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    return contrast, correlation

def compute_cslbp(image):
    radius, n_points = 1, 8
    return local_binary_pattern(image, n_points, radius, method='uniform')

def extract_features(images):
    feature_vectors = []
    for image in images:
        contrast, correlation = glcm_features(image)
        lbp_hist = np.histogram(compute_cslbp(image).ravel(), bins=256, range=(0, 256))[0]
        lbp_hist = lbp_hist / np.sum(lbp_hist)
        feature_vectors.append(np.hstack((contrast, correlation, lbp_hist)))
    return np.array(feature_vectors)

def train_svm(features, labels):
    svm_params = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['rbf']}
    svm = GridSearchCV(SVC(probability=True), svm_params, cv=5)
    svm.fit(features, labels)
    print(f"Best SVM parameters: {svm.best_params_}")
    return svm
    
images, labels = load_data(df)
if images.size == 0:
    raise Exception("No images loaded. Check dataset paths.")

features = extract_features(images)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)

svm_model = train_svm(X_train, y_train)
print("SVM Model Trained Successfully")

accuracy = svm_model.score(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Best SVM parameters: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
SVM Model Trained Successfully
Test Accuracy: 0.7108


In [2]:
import os
import numpy as np
import cv2
from skimage.segmentation import morphological_chan_vese
from skimage.filters import gaussian
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
import matplotlib.pyplot as plt

train_path = "/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Training-20211018T055246Z-001/Training"
test_path = "/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Test-20211018T060000Z-001/Test"

image_size = (256, 256)

def load_images_from_folders(base_path):
    images = []
    labels = []
    
    class_mapping = {'normal': 0, 'glaucoma': 1}
    found_folders = {}

    for root, dirs, files in os.walk(base_path):
        for folder in dirs:
            folder_lower = folder.lower()
            if folder_lower in class_mapping:
                full_path = os.path.join(root, folder)
                found_folders[full_path] = class_mapping[folder_lower]

    if not found_folders:
        raise ValueError(f"No valid class folders ('normal', 'glaucoma') found in {base_path}")

    print(f"Found class folders: {list(found_folders.keys())}")

    for class_path, label in found_folders.items():
        for fname in os.listdir(class_path):
            if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(class_path, fname)
                img = cv2.imread(img_path)
                if img is None:
                    print(f"Warning: Could not read image {img_path}")
                    continue

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, image_size)
                gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

                edges = cv2.Laplacian(gray, cv2.CV_64F)
                edges = cv2.convertScaleAbs(edges) 
                sharpened = cv2.addWeighted(gray, 1.5, edges, -0.5, 0)
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                enhanced = clahe.apply(sharpened)
                normalized = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)

                images.append(normalized.astype(np.uint8))
                labels.append(label)
    
    return np.array(images), np.array(labels)

def segment_optic_disc(image):
    gray = image if len(image.shape) == 2 else cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    smoothed = gaussian(gray, sigma=1)
    segmented_mask = morphological_chan_vese(smoothed, iterations=100, init_level_set="checkerboard")
    return segmented_mask

def glcm_features(image):
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    return contrast, correlation

def compute_cslbp(image):
    radius, n_points = 1, 8
    return local_binary_pattern(image, n_points, radius, method='uniform')

def extract_features(images):
    feature_vectors = []
    for image in images:
        contrast, correlation = glcm_features(image)
        lbp = compute_cslbp(image)
        lbp_hist = np.histogram(lbp.ravel(), bins=256, range=(0, 256))[0]
        lbp_hist = lbp_hist / (np.sum(lbp_hist) + 1e-8)
        feature_vectors.append(np.hstack((contrast, correlation, lbp_hist)))
    return np.array(feature_vectors)

def train_svm(features, labels):
    svm = SVC(C=1, gamma='scale', kernel='rbf', probability=True, class_weight='balanced', random_state=42)
    svm.fit(features, labels)
    return svm

# ==== MAIN EXECUTION ====
images, labels = load_images_from_folders(train_path)
if images.size == 0:
    raise Exception("No images loaded. Check dataset paths.")

print(f"Loaded {len(images)} images.")

features = extract_features(images)

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.15, stratify=labels, random_state=42)

svm_model = train_svm(X_train, y_train)
print("SVM Model Trained Successfully")

accuracy = svm_model.score(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


Found class folders: ['/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Training-20211018T055246Z-001/Training/Images/GLAUCOMA', '/kaggle/input/drishtigs-retina-dataset-for-onh-segmentation/Training-20211018T055246Z-001/Training/Images/NORMAL']
Loaded 50 images.
SVM Model Trained Successfully
Test Accuracy: 0.7500
